## 4.3 로지스틱 회귀(Logistic Regression)

로지스틱 회귀:  반응변수 Y를 직접 모델링하지 않고 Y가 특정 범주에 속하는 확률을 모델링 하는 것.

예제: Defalult자료를 통해 연체확률 모델링.

사용데이터: R - ISLR - Default dataset.

주어진 Balance에 대한 연체확률 : Pr(default = Yes|balance) = p(balance)
이때 p(balance)의 범위는, 0 < p(balance) < 1이 된다
이떄 default낼 것으로 예측하는 데는 보수적일 수록 0에 가깝게 설정하면 된다.

ex) p(balance) > 0.5인 사람은 모두 Default = Yes라고 할 수 있고, \
보수적으로 잡는 다면 p(balance > 0.1과 같이 낮은 임계치를 사용하면 된다.





## 4.3.1 로지스틱 모델
p(X) = Pr(Y = 1|X)와 X간의 관계를 어떻게 모델링 할 것인가에 대한 질문.

4.2절에서는 선형회귀모델을 사용하는 것에 대한 언급이 있었다.

"P(X) = β_0 + β_1X"

이와같은 모델링의 경우 카드대금(balance)이 0에 가까우면 연체확률이 음수, 카드대금이 아주 큰 경우 1보다 커질 수 있다.

따라서, balance가 0과 1 사이에 있다는 전제를 충족시킬 수 없다.

이때 사용하는 것이 로지스틱 함수가 된다.

$$p(X) = \frac {e^{b_0 + b_1X}}{1+e^{b_0 + b_1X}}$$


다음과 같이 표현할 수 있는데,
$$ \frac {p(X)}{1-p(X)} = {e^{b_0 + b_1X}} $$

이때 오른쪽 항을 odds라고 하고, 항상 0과∞사이의 값을 가진다.

이때 공산이 0에 가까울수록 연체확률이 매우 낮고, ∞에 가까울수록 연체확률이 매우 높다는 것을 뜻한다.

ex) \
p(X) = 0.2, odds = 0.2/(1-0.2) = 0.8 = 5명 중 1명이 연체\
p(x) = 0.9, odds = 0.9(1-0.9) = 9 = 10명 중 9명이 연체

$$ \log \frac {p(X)}{1-p(X)} = b_0 + b_1X $$


좌변 : 로그 공산(로짓), 로지스틱 회귀모델은 X에 선형적인 로짓을 가지게 된다.

## 4.3.2. 회귀계수의 추정

$$ b_0, b_1 $$
위의 값은 따로 주어지지 않기 때문에 추정해야 한다.


In [67]:
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm 
import pandas as pd
import numpy as np
import math

In [55]:
url = 'https://raw.githubusercontent.com/RAhyunki/dataset/main/Default.csv' # github에 있는 raw파일 로드
Default = pd.read_csv(url) # credit dataframe에 저장
print(Default.shape)
print(Default.head())

(10000, 4)
  default student      balance        income
0      No      No   729.526495  44361.625074
1      No     Yes   817.180407  12106.134700
2      No      No  1073.549164  31767.138947
3      No      No   529.250605  35704.493935
4      No      No   785.655883  38463.495879


In [56]:
Default.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  object 
 1   student  10000 non-null  object 
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), object(2)
memory usage: 312.6+ KB


In [57]:
Default.default.unique()

array(['No', 'Yes'], dtype=object)

In [58]:
Default['default'] = pd.Series(np.where(Default.default.values == 'Yes', 1, 0), Default.index)
Default['student'] = pd.Series(np.where(Default.student.values == 'Yes', 1, 0), Default.index)
Default.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   default  10000 non-null  int64  
 1   student  10000 non-null  int64  
 2   balance  10000 non-null  float64
 3   income   10000 non-null  float64
dtypes: float64(2), int64(2)
memory usage: 312.6 KB


In [59]:
Default.head()

,default,student,balance,income
0,0,0,729.526495,44361.625074
1,0,1,817.180407,12106.134700
2,0,0,1073.549164,31767.138947
3,0,0,529.250605,35704.493935
4,0,0,785.655883,38463.495879


In [61]:
Default = Default.astype('int')
Default.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   default  10000 non-null  int64
 1   student  10000 non-null  int64
 2   balance  10000 non-null  int64
 3   income   10000 non-null  int64
dtypes: int64(4)
memory usage: 312.6 KB


In [63]:
model_df = sm.Logit.from_formula("default ~ balance", Default)
result_df = model_df.fit()
print(result_df.summary())

Optimization terminated successfully.
         Current function value: 0.079826
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9998
Method:                           MLE   Df Model:                            1
Date:                Tue, 20 Apr 2021   Pseudo R-squ.:                  0.4534
Time:                        04:43:23   Log-Likelihood:                -798.26
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                6.465e-290
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.6481      0.361    -29.493      0.000     -11.356      -9.940
balance        0.0055      0

## 4.3.3 예측하기

계수가 추정되었으면, 임의의 카드대금에 대한 default확률을 계산할 수 있다.

[카드 대금에 대한 default rate]

In [89]:
def default_rate1 (balance):
  exponent = -10.6481 + 0.0055 * balance
  numerator = math.exp(exponent)
  denominator = 1+numerator
  default_rate = numerator/denominator*100
  print("대금이", balance,"$ 인 사람의 연체율 : ", round(default_rate, 2), "%")

In [90]:
default_rate1(1000)

대금이 1000 $ 인 사람의 연체율 :  0.58 %


In [91]:
default_rate1(2000)

대금이 2000 $ 인 사람의 연체율 :  58.71 %


[학생여부에 따른 default rate]

In [92]:
model_df = sm.Logit.from_formula("default ~ student", Default)
result_df = model_df.fit()
print(result_df.summary())

Optimization terminated successfully.
         Current function value: 0.145434
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9998
Method:                           MLE   Df Model:                            1
Date:                Tue, 20 Apr 2021   Pseudo R-squ.:                0.004097
Time:                        04:53:30   Log-Likelihood:                -1454.3
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                 0.0005416
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.5041      0.071    -49.554      0.000      -3.643      -3.366
student        0.4049      0.

In [99]:
def default_rate2 (student):
  exponent = -3.5041 + 0.4049 * student
  numerator = math.exp(exponent)
  denominator = 1+numerator
  default_rate = numerator/denominator*100
  if student == 1:
    print("학생인 경우 연체율은" , round(default_rate, 2), "%")
  else:
    print("학생이 아닌 경우 연체율은" , round(default_rate, 2), "%")

In [102]:
default_rate2(1), default_rate2(0)

학생인 경우 연체율은 4.31 %
학생이 아닌 경우 연체율은 2.92 %


(None, None)

## 4.3.4 다중로지스틱 회귀


$$ \log \frac {p(X)}{1-p(X)} = b_0 + b_1X + ... + b_pX_p $$

$$p(X) = \frac {e^{b_0 + b_1X + ... +b_pX_p  }}{1+e^{b_0 + b_1X + ... +b_pX_p}}$$

[대금, 소득, 학생여부에 따른 default rate]

In [103]:
model_df = sm.Logit.from_formula("default ~ balance + income + student", Default)
result_df = model_df.fit()
print(result_df.summary())

Optimization terminated successfully.
         Current function value: 0.078581
         Iterations 10
                           Logit Regression Results                           
Dep. Variable:                default   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9996
Method:                           MLE   Df Model:                            3
Date:                Tue, 20 Apr 2021   Pseudo R-squ.:                  0.4619
Time:                        05:01:18   Log-Likelihood:                -785.81
converged:                       True   LL-Null:                       -1460.3
Covariance Type:            nonrobust   LLR p-value:                3.389e-292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -10.8658      0.492    -22.078      0.000     -11.830      -9.901
balance        0.0057      0

In [113]:
def default_rate2 (balance, income, student):
  income1 = income/1000
  exponent = -10.8658 + 0.0057 * balance + 0.003 * income1 -0.6468 * student  
  numerator = math.exp(exponent)
  denominator = 1+numerator
  default_rate = numerator/denominator*100
  if student == 1:
    print("카드대금이", balance, "$이고, 소득이", income,"$인 학생인 경우 연체율은" , round(default_rate, 2))
  else:
    print("카드대금이", balance, "$이고, 소득이", income,"$인 학생이 아닌 경우 연체율은" , round(default_rate, 2))

In [115]:
default_rate2(1500, 40000, 1), default_rate2(1500, 40000, 0)

카드대금이 1500 $이고, 소득이 40000 $인 학생인 경우 연체율은 5.51
카드대금이 1500 $이고, 소득이 40000 $인 학생이 아닌 경우 연체율은 10.01


(None, None)